## icmecat

makes the ICMECATv2.0

Author: C. Moestl, IWF Graz, Austria
twitter @chrisoutofspace, part of https://github.com/cmoestl/heliocats
last update March 2020

Install a specific conda environment to run this code, see https://github.com/cmoestl/heliocats

**current status: work in progress**

* to do: despike sta stb wind all, new STA, Wind and PSP converted to SCEQ components 

* Convert to script with 

    jupyter nbconvert --to script icmecat.ipynb
    
    
* Adding a new event: edit the file icmecat/HELCATS_ICMECAT_v20_master.xlsx to add 3 times, the id and spacecraft name, 
delete the file for the respective spacecraft under data/indices_icmecat, and run this notebook or script.


In [61]:
import numpy as np
import scipy.io
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import  DateFormatter
import seaborn as sns
import datetime
import astropy.constants as const
from sunpy.time import parse_time
import time
import pickle
import sys
import os
import urllib
import json
import importlib
import pandas as pd
import copy
import openpyxl


from heliocats import plot as hp
importlib.reload(hp) #reload again while debugging

from heliocats import data as hd
importlib.reload(hd) #reload again while debugging

from heliocats import cats as hc
importlib.reload(hc) #reload again while debugging

#where the 6 in situ data files are located is read from input.py
#as data_path=....
from input import data_path


########### make directories first time if not there

resdir='results'
if os.path.isdir(resdir) == False: os.mkdir(resdir)

datadir='data'
if os.path.isdir(datadir) == False: os.mkdir(datadir)

indexdir='data/indices_icmecat' 
if os.path.isdir(indexdir) == False: os.mkdir(indexdir) 


catdir='icmecat'
if os.path.isdir(catdir) == False: os.mkdir(catdir)

icplotsdir='icmecat/plots_icmecat/' 
if os.path.isdir(icplotsdir) == False: os.mkdir(icplotsdir) 

### (1) load new data with HelioSat and heliocats.data

In [62]:
   
load_data=1

if load_data > 0:

    print('load new Wind, STEREO-A, MAVEN, Parker Solar Probe data')

    # MAVEN
    #filemav='maven_2014_2018.p'
    #[mav,hmav]=pickle.load(open(filemav, 'rb' ) )

    #filemav='maven_2014_2018_removed.p'
    #[mav,hmav]=pickle.load(open(filemav, 'rb' ) )
    
    filemav='maven_2014_2018_removed_smoothed.p'
    [mav,hmav]=pickle.load(open(data_path+filemav, 'rb' ) )

    # Wind
    filewin="wind_2018_2019.p" 
    #for updating data
    #start=datetime.datetime(2018, 1, 1)
    #end=datetime.datetime.utcnow()
    #hd.save_wind_data(data_path,filewin,start,end)
    [win2,hwin2]=pickle.load(open(data_path+filewin, "rb" ) )  

    # STEREO-A    
    filesta2='sta_2018_2019_beacon.p'
    #start=datetime.datetime(2018, 1, 1)
    #end=datetime.datetime(2019, 12, 31)
    #hd.save_stereoa_beacon_data(data_path,filesta,start,end)
   
    [sta2,hsta2]=pickle.load(open(data_path+filesta2, "rb" ) )  

    # Parker Solar Probe
    filepsp='psp_2018_2019.p'
    [psp,hpsp]=pickle.load(open(data_path+filepsp, "rb" ) )  


    # ADD BepiColombo  
    
    
    # ADD Solar Orbiter
    
    
    # Ulysses is currently taken from the full 
    # helcats data below, but a file is available on figshare


    # get data file from helcats with headers
    [vex,win,mes,sta,stb,uly,hvex,hwin,hmes,hsta,hstb,huly]=hd.load_helcats_datacat(data_path+'helcats_all_data_removed.p') 
    print('done')

load new Wind, STEREO-A, MAVEN, Parker Solar Probe data
load all helcats DATACAT from single file:  /nas/helio/data/insitu_python/helcats_all_data_removed.p
Use vex,win,sta,stb,mes,uly to access data and position, hvex,hwin, hmes, hsta, hstb, huly for headers.
done


## (2) measure new events 

In [63]:
#for measuring new events use this function from heliocats.plot 
#plt.close('all')
#works in jupyter notebooks

#works in scripts
#matplotlib.use('qt5agg')  
#plt.ion()
#hp.plot_insitu_measure(psp, '2018-Nov-10','2018-Nov-15', 'PSP', 'results/plots_icmecat/')
#%matplotlib 
#hp.plot_insitu_measure(win, '2010-Oct-29','2010-Oct-31', 'Wind', 'results/')


#for plotting single events
#hp.plot_insitu(psp, ic.icme,'2018-Nov-15', 'PSP', icplotsdir)

### (3) make ICMECAT 

In [64]:
print('data loaded')
ic=hc.load_helcats_icmecat_master_from_excel('icmecat/HELCATS_ICMECAT_v20_master.xlsx')

####### 3a get indices for all spacecraft

wini=np.where(ic.sc_insitu == 'Wind')[:][0] 
stai=np.where(ic.sc_insitu == 'STEREO-A')[:][0]    
stbi=np.where(ic.sc_insitu == 'STEREO-B')[:][0]    
vexi=np.where(ic.sc_insitu == 'VEX')[:][0]  
mesi=np.where(ic.sc_insitu == 'MESSENGER')[:][0]   
ulyi=np.where(ic.sc_insitu == 'ULYSSES')[:][0]    
mavi=np.where(ic.sc_insitu == 'MAVEN')[:][0]    
pspi=np.where(ic.sc_insitu == 'PSP')[:][0]    


####### 3b get parameters for all spacecraft one after another

ic=hc.get_cat_parameters(win,wini,ic,'Wind')
ic=hc.get_cat_parameters(sta,stai,ic,'STEREO-A')
ic=hc.get_cat_parameters(stb,stbi,ic,'STEREO-B')
ic=hc.get_cat_parameters(vex,vexi,ic,'VEX')
ic=hc.get_cat_parameters(mes,mesi,ic,'MESSENGER')
ic=hc.get_cat_parameters(uly,ulyi,ic,'ULYSSES')
ic=hc.get_cat_parameters(mav,mavi,ic,'MAVEN')
ic=hc.get_cat_parameters(psp,pspi,ic,'PSP')


####### 3c make all plots if wanted
matplotlib.use('Agg')
hp.plot_icmecat_events(win,wini,ic,'Wind',icplotsdir)
hp.plot_icmecat_events(sta,stai,ic,'STEREO-A',icplotsdir)
hp.plot_icmecat_events(stb,stbi,ic,'STEREO-B',icplotsdir)
hp.plot_icmecat_events(vex,vexi,ic,'VEX',icplotsdir)
hp.plot_icmecat_events(mes,mesi,ic,'MESSENGER',icplotsdir)
hp.plot_icmecat_events(uly,ulyi,ic,'ULYSSES',icplotsdir)
hp.plot_icmecat_events(mav,mavi,ic,'MAVEN',icplotsdir)
hp.plot_icmecat_events(psp,pspi,ic,'PSP',icplotsdir)


data loaded
load HELCATS ICMECAT from file: icmecat/HELCATS_ICMECAT_v20_master_nonew.xlsx
Get parameters for  Wind


/home/cmoestl/pycode/heliocats/heliocats/cats.py:236: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'sheath_speed_mean']=np.round(np.nanmean(sc.vt[icme_start_ind[i]:mo_start_ind[i]]),1)
/home/cmoestl/miniconda/envs/helio/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:239: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'sheath_density_mean']=np.round(np.nanmean(sc.np[icme_start_ind[i]:mo_start_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:244: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'sheath_pdyn_mean']=np.round(np.nanmean(pdyn_i),1)


Get parameters for  STEREO-A


/home/cmoestl/pycode/heliocats/heliocats/cats.py:172: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'icme_speed_mean']=np.round(np.nanmean(sc.vt[icme_start_ind[i]:mo_end_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:218: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'mo_speed_mean']=np.round(np.nanmean(sc.vt[mo_start_ind[i]:mo_end_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:223: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'mo_density_mean']=np.round(np.nanmean(sc.np[mo_start_ind[i]:mo_end_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:226: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'mo_temperature_mean']=np.round(np.nanmean(sc.tp[mo_start_ind[i]:mo_end_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:231: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'mo_pdyn_mean']=np.round(np.nanmean(pdyn_i),1)


Get parameters for  STEREO-B
Get parameters for  VEX
Get parameters for  MESSENGER
Get parameters for  ULYSSES
Get parameters for  MAVEN
Get parameters for  PSP
0
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20070114_01.png
1
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20070115_01.png
2
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20070329_01.png
3
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20070521_01.png
4
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20070608_01.png
5
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20071119_01.png
6
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20071225_01.png
7
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20080523_01.png
8
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20080903_01.png
9
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20080917_01.png
10
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20081204_01.png
11
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_20081217_01.png
12
saved as  icmecat/plots_icmecat/ICME_Wind_NASA_2009010

### (4) save ICMECAT 

In [ ]:

header='ICME CATALOGUE v2.0 \n\n\
This is the HELCATS interplanetary coronal mass ejection (ICME) catalog, based on in situ magnetometer and plasma observations in the heliosphere. \n\n\
This is version 2.0, released in March 2020 with major update to the original version 1.0, originally a product of EU HELCATS project (2014-2017). \n\n\
Released 2020-??-??. DOI: 10.6084/m9.figshare.4588315.v2 \n\n\
Number of events in ICMECAT: '+str(len(ic))+' \n\n\
ICME observatories: Wind, Parker Solar Probe, STEREO-A, STEREO-B, Venus Express, MESSENGER, MAVEN, ULYSSES \n\n\
Time range: January 2007 - December 2018. \n \n\
Authors: Christian Moestl, Andreas Weiss, Space Research Institute, Austrian Academy of Sciences, Graz, Austria,\n\
Contributors: Peter Boakes, Alexey Isavnin, Emilia Kilpua, Reka Winslow, Brian Anderson, Lydia Philpott,\n\
Vratislav Krupar, Jonathan Eastwood, Simon Good, Lan Jian, Teresa Nieves-Chinchilla.  \n\n\
The catalog has been made by getting 3 times of the ICME (shock or disturbance begin, \
magnetic obstacle start and end) from these individual catalogs and then calculating all parameters again consistently from all data: \n\n\
Wind: Nieves-Chinchilla et al. 2018: \n\
STEREO-A, STEREO-B Lan Jian ....\n\
VEX:  Good et al. 2018 \n\
MESSENGER:  Good et al. 2018 Winslow et al. 2018 \n\
MAVEN: Guo et al. (note that this is a mixture of methods MSL RAD, MAVEN and STEREO HI, Möstl et al. 2020)\n\
Additionally we have added individual events at Ulysses,STEREO-B, STEREO-A, Parker Solar Probe, Wind and MAVEN.\n\n'


parameters='1: ICMECAT_ID: The unique identifier for the observed ICME. unit: string. \n\
3: ICME_START_TIME: The shock arrival or density enhancement time, can be similar to MO_START_TIME. unit: UTC. \n\
4: MO_START_TIME: The start time of the magnetic obstacle (MO), including flux ropes, flux-rope-like, and ejecta signatures. unit: UTC. \n\
5: MO_END_TIME: The end time of the magnetic obstacle. unit: UTC. \n\
2: MO_SC_HELIODISTANCE: Average heliocentric distance of the spacecraft during the MO. unit: AU.\n\
3: MO_LONG_HEEQ: Average heliospheric longitude of the spacecraft during the MO, range [-180,180]. unit: degree (HEEQ).\n\
4: MO_LAT_HEEQ: Average heliospheric latitude of the spacecraft during the MO, range [-90,90]. unit: degree (HEEQ).\n\
2: SC_INSITU: The name of the in situ observatory. unit: string. \n\
7: MO_BMAX: The maximum total magnetic field in the magnetic obstacle. unit: nT.\n\
8: MO_BMEAN: The mean total magnetic field of the magnetic obstacle. unit: nT.\n\
9: MO_BSTD: The standard deviation of the total magnetic field of the magnetic obstacle. unit: nT.\n\
10: MO_BZMEAN: The mean magnetic field Bz component in the magnetic obstacle. unit: nT.\n\
11: MO_BZMIN: The minimum magnetic field Bz component of the magnetic obstacle. unit: nT.\n\
12: MO_DURATION: Duration of interval between MO_START_TIME and MO_END_TIME. unit: hours.\n\
19: SHEATH_SPEED: For STEREO-A/B, Wind, MAVEN: average proton speed from ICME_START_TIME to MO_START_TIME, NaN if these times are similar. unit: km/s.\n\
20: SHEATH_SPEED_STD: For STEREO-A/B, Wind, MAVEN: standard deviation of proton speed from ICME_START_TIME to MO_START_TIME, NaN if these times are similar. unit: km/s.\n\
21: MO_SPEED: For STEREO-A/B, Wind, MAVEN: average proton speed from MO_START_TIME to MO_END_TIME. unit: km/s.\n\
22: MO_SPEED_STD: For STEREO-A/B, Wind, MAVEN: standard deviation of proton speed from MO_START_TIME to MO_END_TIME. unit: km/s.\n\
23: SHEATH_DENSITY: For STEREO-A/B, Wind, MAVEN: average proton density from ICME_START_TIME to MO_START_TIME, NaN if these times are similar. unit: ccm^-3.\n\
24: SHEATH_DENSITY_STD: For STEREO-A/B, Wind, MAVEN: standard deviation of proton density from ICME_START_TIME to MO_START_TIME, NaN if these times are similar. unit: cm^-3.\n\
25: MO_DENSITY: For STEREO-A/B, Wind, MAVEN: average proton density from MO_START_TIME to MO_END_TIME. unit: cm^-3.\n\
26: MO_DENSITY_STD: For STEREO-A/B, Wind, MAVEN: standard deviation of proton density from MO_START_TIME to MO_END_TIME. unit: cm^-3.\n\
27: SHEATH_TEMPERATURE: For STEREO-A/B, Wind, MAVEN:average proton temperature from ICME_START_TIME to MO_START_TIME, NaN if these times are similar. unit: K.\n\
28: SHEATH_TEMPERATURE_STD: For STEREO-A/B, Wind, MAVEN: standard deviation of proton temperature from ICME_START_TIME to MO_START_TIME, NaN if these times are similar. unit: K.\n\
29: MO_TEMPERATURE: For STEREO-A/B, Wind, MAVEN: average proton temperature from MO_START_TIME to MO_END_TIME. unit: K.\n\
30: SHEATH_PDYN_MEAN: For STEREO-A/B, Wind, MAVEN: mean of dynamic pressure assuming only protons contribute, from ICME_START_TIME to MO_START_TIME. unit: nPa.\n\
31: SHEATH_PDYN_STD: For STEREO-A/B, Wind, MAVEN: standard deviation of dynamic pressure assuming only protons contribute, from ICME_START_TIME to MO_START_TIME. unit: nPa.\n\
32: MO_PDYN_MEAN: For STEREO-A/B, Wind, MAVEN: mean of dynamic pressure assuming only protons contribute, from MO_START_TIME to MO_END_TIME. unit: nPa.\n\
33: MO_PDYN_STD: For STEREO-A/B, Wind, MAVEN: standard deviation of dynamic pressure assuming only protons contribute, from MO_START_TIME to MO_END_TIME. unit: nPa.\n'



header_html='<p> ICME CATALOGUE v2.0 <br /> <br /> \
This is the HELCATS interplanetary coronal mass ejection (ICME) catalog, based on in situ magnetometer and plasma observations in the heliosphere.<br /> <br /> \
Version 2.0, released in March 2020 with major update to the original version 1.0, originally a product of EU HELCATS project (2014-2017). <br /><br />  \
Number of events in ICMECAT: '+str(len(ic))+' <br /> <br /> \
ICME observatories: Wind, Parker Solar Probe, STEREO-A, STEREO-B, Venus Express, MESSENGER, MAVEN, ULYSSES <br /> \
Authors: Christian Moestl, Andreas Weiss, Space Research Institute, Austrian Academy of Sciences, Graz, Austria,<br /> \
Contributors: Peter Boakes, Alexey Isavnin, Emilia Kilpua, Reka Winslow, Brian Anderson, Lydia Philpott,<br /> \
Vratislav Krupar, Jonathan Eastwood, Simon Good, Lan Jian, Teresa Nieves-Chinchilla. <br /> <br /> \
Time range: January 2007 - December 2018. <br /> <br /> \
This is version: 2.0 of the catalogue, released 2020-??-??. DOI: 10.6084/m9.figshare.4588315.v2 <p>'


parameters_html=''



print(header)
print(parameters)

#make header
#header=hc.make_icmecat_header(ic)
file='icmecat/HELCATS_ICMECAT_v20_header.txt'
with open(file, "w") as text_file:
    text_file.write(header)

print()    
print()    
print()    


print('header saved as '+file)

In [12]:
### save ICMECAT as pickle with times as datetime objects
file='icmecat/HELCATS_ICMECAT_v20.p'
pickle.dump(ic, open(file, 'wb'))
print('ICMECAT saved as '+file)



################ save to different formats

#copy pandas dataframe first to change time format

ic_copy=copy.deepcopy(ic)  
ic_copy.icme_start_time=parse_time(ic.icme_start_time).isot
ic_copy.mo_start_time=parse_time(ic.mo_start_time).isot
ic_copy.mo_end_time=parse_time(ic.mo_end_time).isot

#change time format
for i in np.arange(len(ic)):

    dum=ic_copy.icme_start_time[i] 
    ic_copy.at[i,'icme_start_time']=dum[0:16]
     
    dum=ic_copy.mo_start_time[i] 
    ic_copy.at[i,'mo_start_time']=dum[0:16]
     
    dum=ic_copy.mo_end_time[i] 
    ic_copy.at[i,'mo_end_time']=dum[0:16]


#save as Excel
file='icmecat/HELCATS_ICMECAT_v20.xlsx'
ic_copy.to_excel(file,sheet_name='ICMECATv2.0')
print('ICMECAT saved as '+file)

#save as json
file='icmecat/HELCATS_ICMECAT_v20.json'
ic_copy.to_json(file)
print('ICMECAT saved as '+file)

#save as csv
file='icmecat/HELCATS_ICMECAT_v20.csv'
ic_copy.to_csv(file)
print('ICMECAT saved as '+file)

#save as html
file='icmecat/HELCATS_ICMECAT_v20_simple.html'
ic_copy.to_html(file)
print('ICMECAT saved as '+file)

#save as hdf needs pip install tables
#file='icmecat/HELCATS_ICMECAT_v20.hdf'
#ic.to_hdf(file,key='icmecat')

#save as .mat does not work yet
#ile='icmecat/HELCATS_ICMECAT_v20.mat'
#icdict=ic.to_dict()
#scipy.io.savemat(file,ic.values)

#save as txt
file='icmecat/HELCATS_ICMECAT_v20.txt'
np.savetxt(file, ic_copy.values.astype(str), fmt='%s' )

print('ICMECAT saved as '+file)

ic2=hc.load_icmecat()



############ save as html file for helioforecast.space

file='icmecat/HELCATS_ICMECAT_v20.p'
ic=pickle.load( open(file, 'rb'))

#save as html
file='icmecat/HELCATS_ICMECAT_v20.html'
#ic.to_html(file,justify='center')

ichtml='{% extends "_base.html" %} \n \n {% block content %} \n \n \n '
ichtml  += header_html
ichtml += ic.to_html()
ichtml +='\n \n {% endblock %}'

'''
{% extends "_base.html" %}
{% block content %}
<div class="row">
    <div class="col-12 col-md-12 col-xl-12">
        <p> </br>
        Recent real-time solar and solar wind data from SDO in Earth orbit, 
        from the DSCOVR or ACE spacecraft at the Sun-Earth L1 point provided by NOAA and from STEREO-A.
        Updated twice daily at 05:00 and 17:00 UTC.</br></br>
        </p>
    </div>
</div>
'''



with open(file,'w') as f:
    f.write(ichtml)
    f.close()
    
print('ICMECAT saved as '+file)    

ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.p
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.xlsx
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.json
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.csv
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20_simple.html
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.txt
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.html
